<a href="https://colab.research.google.com/github/timthedev07/dog-breed-recognition/blob/dev/dog_breed_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load the dataset

In [1]:
!rm -rf ./*
!mkdir -p data
!rm -rf sample_data
%cd data
!wget 'https://storage.googleapis.com/kagglesdsdata/competitions/7327/861871/labels.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1660579199&Signature=aXTJAR3DswCBF8iEBuo6imc%2BYBOXJ5IuoZv1%2F4vFixkAGRElW0vwnMi0TAnJKpwv%2B0%2Fd6cNPEztGZJKlcA0tiizcVFy8P0QV5a2VGq23TfY8fUPV1JC1w2zpxmegBbyw%2F%2By9Pj2Dgf36YVnZzAGPbPkxWWx9l%2BOyP%2FSBa4381z7Ok5WGwi1bx0nZ7IkYnvAr4CCGz2VOvXebZ3mGQc4g%2ByxDX40qkKerpNvT%2FBQt7gyLaRwiZ7l8ncLNlK1bwH7OMdavO51Z%2F26s6eTj%2FWSyp5JY6A0igxEybJkCqQKErKvTlK%2Fhjh%2Bu1nWre1BC2j8m6WT5E8sXXnrEHVnN1mS8pg%3D%3D&response-content-disposition=attachment%3B+filename%3Dlabels.csv' -O labels.csv
!mkdir -p images
%cd images
!wget 'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/7327/861871/upload/train.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1660577719&Signature=OwyFagYt8OLDWupAQF%2FkSAMsab17M4nGHk%2BxQ2Vxh7TuE5u26LzbdY8nro8gF7bbsIN5k235K4Ni2%2F9r5FWnQ23LrFXf9SvKwqWcJqhZvplq3uPK7KfqCz7HUisW5dNyEuyXDYqs%2Fm6ghWgLV9f0ob1mUdMfFb0MgWPTSE47iYXogZoam4fTJ3p44hHiJHXj2gI2GD%2BoSWwfhw94zi0%2FLG%2FfKU31KEiUHt6F%2Ftprbf8mR8NEGqfIEBTWEs4xKGMX14m631tGi4DdOnSaA5IrvZL4ZYIUpTZRO5JSbVS7hCTSuhQqZIvl%2B9%2FGWbuZfaMRqwy1nO%2FBjr6XK5CGU5Mg%2Fw%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.zip' -O "train.zip"
!unzip -q train.zip
!rm -rf train.zip
%cd ../..

/content/data
--2022-08-12 16:19:32--  https://storage.googleapis.com/kagglesdsdata/competitions/7327/861871/labels.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1660579199&Signature=aXTJAR3DswCBF8iEBuo6imc%2BYBOXJ5IuoZv1%2F4vFixkAGRElW0vwnMi0TAnJKpwv%2B0%2Fd6cNPEztGZJKlcA0tiizcVFy8P0QV5a2VGq23TfY8fUPV1JC1w2zpxmegBbyw%2F%2By9Pj2Dgf36YVnZzAGPbPkxWWx9l%2BOyP%2FSBa4381z7Ok5WGwi1bx0nZ7IkYnvAr4CCGz2VOvXebZ3mGQc4g%2ByxDX40qkKerpNvT%2FBQt7gyLaRwiZ7l8ncLNlK1bwH7OMdavO51Z%2F26s6eTj%2FWSyp5JY6A0igxEybJkCqQKErKvTlK%2Fhjh%2Bu1nWre1BC2j8m6WT5E8sXXnrEHVnN1mS8pg%3D%3D&response-content-disposition=attachment%3B+filename%3Dlabels.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 142.250.141.128, 2607:f8b0:4023:c0d::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 482063 (471K) [application/octet-stream]
Saving to: ‘labels.csv’

labels.csv

Building and training the machine learning model.

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.models import Sequential as SequentialType
from sklearn.model_selection import train_test_split
import termcolor as tc

csvFname = "labels.csv"

pathJoin = os.path.join

DATA_DIR = "data"
IMG_DIR = "images"

class DogBreedModel:
    def __init__(self, trainPercentage = 80, production = False, dataSize = None) -> None:
        """
        Params:
          - `trainPercentage`% indicates how much of the given data should be used for **training**
          - `dataSize` indicates how much of the downloaded data should be used, leave as None if all data should be involved
        """
        self.trainPercentage = trainPercentage
        self.BATCH_SIZE = 32
        self.dataSize = dataSize

        self.labels = []
        self.model: SequentialType = None
        self.RESIZED_IMG_WIDTH = 224
        self.RESIZED_IMG_HEIGHT = 224

        self.labelsData = pd.DataFrame()

        # train/test data
        self.trainX = []
        self.trainY = []
        self.testX = []
        self.testY = []

        # storing the classifier
        self.classifier: SequentialType = Sequential([])

        self.populateLabels()
        self.initClassifier()

        if not production:
            self.loadDataset()
            self.initModel()
        else:
            self.loadModel()

    def getImgLabelPair(self, fileName: str, label):
        return self.imgToNp(fileName), label

    def createDataBatches(self, X, y=None, validation = False):
        if validation:
            validationData = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
            return validationData.map(self.getImgLabelPair).batch(self.BATCH_SIZE)
        else:
            trainData = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y))).shuffle(len(X))
            return trainData.map(self.getImgLabelPair).batch(self.BATCH_SIZE)

    def initClassifier(self):
        """
        Initializing the pre-trained resnet 50 v2 model.

        Call this before other initialization methods.
        """
        classifier = tf.keras.applications.mobilenet_v2.MobileNetV2(
            input_shape=(self.RESIZED_IMG_WIDTH, self.RESIZED_IMG_HEIGHT, 3),
            include_top = False,
            weights='imagenet',
            classes=len(self.labels)
        )
        for layer in classifier.layers:
            layer.trainable = False

        self.classifier = classifier

        print(tc.colored("Classifier model initialized.", "green"))

    def populateLabels(self):
        labelsInfo = pd.read_csv(pathJoin(DATA_DIR, csvFname))
        breeds = labelsInfo["breed"].unique()
        breeds.sort()

        self.labels = breeds

        print(tc.colored("Labels populated.", "green"))

    def yDataOneHot(self, y: np.ndarray):
        """
        y should be a 1-dimensional array containing the breeds as strings
        """
        return np.array([(label == self.labels).astype(int) for label in y])

    def xNormalize(self, x: np.ndarray):
        """
        turns all pixel values in all images provided into a decimal in the range of [0, 1]
        """
        return tf.image.convert_image_dtype(x, tf.float32)

    def loadDataset(self):
        print(tc.colored("Loading dataset & labels...", "yellow"))

        csvData = pd.read_csv(pathJoin(DATA_DIR, csvFname))
        imgFilenames = csvData["id"]
        n = self.dataSize if self.dataSize is not None else len(imgFilenames)
        imgFilenames = imgFilenames[:n].map(lambda x: pathJoin(DATA_DIR, IMG_DIR, x + ".jpg"))
        print(tc.colored("  Image file names loaded.", "green"))

        allY = self.yDataOneHot(csvData["breed"].to_numpy()[:n])
        print(tc.colored("  Labels one-hot encoded.", "green"))

        self.trainX, self.testX, self.trainY, self.testY = train_test_split(imgFilenames, allY, test_size=(1 - (self.trainPercentage / 100)), shuffle=False)
        print(tc.colored("Dataset & labels loaded.", "green"))

    def imgToNp(self, fileName: tf.Tensor):
        """
        Reads a given image, resize it, normalize it, and converts it to a tf tensor

        The `fileName` param should be a string tensor
        """
        file = tf.io.read_file(fileName)
        img = tf.io.decode_image(file, channels = 3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize_with_crop_or_pad(img, 224, 224)
        return img

    def initModel(self):
        """
        Initialized an untrained model defined with an architecture
        """
        self.model = Sequential([
            self.classifier,
            BatchNormalization(),
            GlobalAveragePooling2D(),
            Dropout(0.3),
            Dense(len(self.labels), activation="softmax")
        ])

        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

        print(tc.colored("Model compiled.", "green"))

    def trainModel(self):
        callback = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=2,
            baseline=None,
            restore_best_weights=True
        )

        print(tc.colored("Creating data batches for training...", "yellow"))
        trainData = self.createDataBatches(self.trainX, self.trainY)
        validationData = self.createDataBatches(self.testX, self.testY, validation=True)
        print(tc.colored("Data batches created.", "green"))

        self.model.fit(
            trainData,
            steps_per_epoch = len(trainData),
            epochs = 10,
            callbacks = [callback],
        )
        self.model.evaluate(validationData)
        self.saveModel()

    def loadModel(self, path = "model"):
        """
        When an instance is first initialized, use this method to load
        an entire model already saved somewhere(if applicable).

        `path`: path to the directory of the saved model
        """
        loaded: SequentialType = tf.keras.models.load_model(path)
        self.model = loaded
        print(tc.colored("Model loaded.", "green"))

    def saveModel(self, path = "model"):
        """
        Save a trained model
        """
        self.model.save(path)
        print(tc.colored("Model saved.", "green"))

    def predict(self, image: np.ndarray) -> str:
        [prediction] = self.model.predict([image])

        print(prediction)
        # for key, val in self.labels.values():
        #     if val == outputLabelNum:
        #         return key


def main():
    model = DogBreedModel()
    model.trainModel()

if __name__ == "__main__":
    main()

Labels populated.
9420800/9406464 [==============================] - 0s 0us/step
Classifier model initialized.
Loading dataset & labels...
  Image file names loaded.
  Labels one-hot encoded.
Dataset & labels loaded.
Model compiled.
Creating data batches for training...
Data batches created.
Epoch 1/10
256/256 [==============================] - ETA: 0s - loss: 2.3521 - accuracy: 0.4575

256/256 [==============================] - 231s 875ms/step - loss: 2.3521 - accuracy: 0.4575
Epoch 2/10
 81/256 [========>.....................] - ETA: 2:29 - loss: 1.0682 - accuracy: 0.7149